In [31]:
import pandas as pd
from src.data import load_data

train, test = load_data()
df = pd.concat([train, test])
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


1. impute

In [33]:
import importlib
import src.features as features

features = importlib.reload(features)   # reload the module
df = features.impute(df)

2. encode

In [88]:
from src.features import preprocess, evaluate_model, unordered_categorical_features
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from pandas.api.types import CategoricalDtype
from xgboost import XGBRegressor

# start with ordered categorical features

ordered_features = [
    "LotShape", "Utilities", "LandSlope", "OverallQual", "OverallCond", "ExterQual", "ExterCond",
    "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "KitchenQual",
    "Functional", "FireplaceQu", "GarageFinish", "GarageQual", "GarageCond", "PavedDrive",
    "PoolQC", "Fence"
]

five_levels = list(range(1, 6))
ten_levels = list(range(1, 11))

ordered_levels_num = {
    "OverallQual": ten_levels,
    "OverallCond": ten_levels,
    'GarageQual': five_levels,
    'ExterQual': five_levels,
    'ExterCond': five_levels,
    'BsmtQual': five_levels,
    'BsmtCond': five_levels,
    'HeatingQC': five_levels,
    'KitchenQual': five_levels,
    'FireplaceQu': five_levels,
    'GarageCond': five_levels,
    'PoolQC': five_levels,
}
ordered_levels_str = {
    'Fence': ['MnWd', 'GdWo', "MnPrv", 'GdPrv'],
    'PavedDrive': ['N', 'P', 'Y'],
    "Utilities": ["NoSeWa", "NoSewr", "AllPub"],
    "CentralAir": ["N", "Y"],
    "Electrical": ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"],
    "LotShape": ["Reg", "IR1", "IR2", "IR3"],
    "LandSlope": ["Sev", "Mod", "Gtl"],
    "BsmtExposure": ["No", "Mn", "Av", "Gd"],
    "BsmtFinType1": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "BsmtFinType2": ["Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"],
    "Functional": ["Sal", "Sev", "Maj1", "Maj2", "Mod", "Min2", "Min1", "Typ"],
    "GarageFinish": ["Unf", "RFn", "Fin"]
}

# add a 'None' at the beginning for missing values
ordered_levels = {key: ['None'] + value for key, value in ordered_levels_str.items()}
ordered_levels = {key: [0] + value for key, value in ordered_levels_num.items()}

ordered_cols_num = list(ordered_levels_num.keys())
ordered_cols_str = list(ordered_levels_str.keys())

for name, levels in ordered_levels.items():
    if name in df.columns:
        df[name] = df[name].astype(CategoricalDtype(levels, ordered=True))

for name in unordered_categorical_features:
        df[name] = df[name].astype('category')
        if 'None' not in df[name].cat.categories:
            df[name] = df[name].cat.add_categories('None')

# Establish Baseline

In [91]:
df_train = df.loc[train.index, :]
df_test = df.loc[test.index, :]

X_train = df_train.drop(columns=["SalePrice", "Id"], errors="ignore").copy()
y_train = df_train["SalePrice"].copy()
X_test = df_test.drop(columns=["SalePrice", "Id"], errors="ignore").copy()
y_test = df_test['SalePrice'].copy()

preprocessor = preprocess(X_train, ordered_levels)
ridge_model = Pipeline(steps=[
    ("prep", preprocessor),
    ("model", Ridge(alpha=1.0))
])

# CV baseline
scores = evaluate_model(X_train, y_train, model)
print(scores)

xgb_params = dict(
    max_depth=6,           # maximum depth of each tree - try 2 to 10
    learning_rate=0.01,    # effect of each tree - try 0.0001 to 0.1
    n_estimators=1000,     # number of trees (that is, boosting rounds) - try 1000 to 8000
    min_child_weight=1,    # minimum number of houses in a leaf - try 1 to 10
    colsample_bytree=0.7,  # fraction of features (columns) per tree - try 0.2 to 1.0
    subsample=0.7,         # fraction of instances (rows) per tree - try 0.2 to 1.0
    reg_alpha=0.5,         # L1 regularization (like LASSO) - try 0.0 to 10.0
    reg_lambda=1.0,        # L2 regularization (like Ridge) - try 0.0 to 10.0
    num_parallel_tree=1,   # set > 1 for boosted random forests
)

xgb_model = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', XGBRegressor(**xgb_params))
])
scores = evaluate_model(X_train, y_train, xgb_model)
print(scores)

/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase

{'cv_rmsle': '0.15587', 'cv_mae': '21987.42024'}


/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase

{'cv_rmsle': '0.13461', 'cv_mae': '15921.31775'}


/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)
/Users/alyeltayeb/Library/Mobile Documents/com~apple~CloudDocs/Phase 2/.venv/lib/python3.9/site-packages/pandas/core/arrays/categorical.py:1692: RuntimeWarning: invalid value encountered in cast
  return np.asarray(ret, dtype=dtype)


_ridge scores_:

cv_rmsle: 0.15587, cv_mae: 21987.42024

<br>

_xgboost scores_:

cv_rmsle: 0.13461, cv_mae: 15921.31775

<br>

**XGBOOST** model performed a lot better than Ridge, so we proceed with XGBRegressor.

Will test more models later on.